In [0]:
"""
SILVER LAYER
- batch join
- will work on streaming requirement in next iteration
"""

def process_silver(source_table, target_table):
    print(f"Processing {target_table}")

    # Read tables
    weather_df = spark.read.table(source_table)
    mapping_df = spark.read.table("bronze_weather_mapping") \
        .drop("_ingestion_time", "_source_file")
    
    # Join weather with the mapping table to get 'county'
    enriched_df = weather_df.join(
        mapping_df,
        on=["latitude", "longitude"],
        how="left"
    )
    
    # Write to delta
    enriched_df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(target_table)
        
    print(f"Created {target_table}")

process_silver("bronze_weather_hist", "silver_weather_hist")
process_silver("bronze_weather_forecast", "silver_weather_forecast")